In [1]:
from google.cloud import bigquery
#import polars as pl
import pandas as pd
#import plotnine as p9
#from pprint import pprint

In [2]:
# Set up a client
client = bigquery.Client(project = "umt-msba")

c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [9]:
# Write your query
query = """
    SELECT DISTINCT card_no
    FROM `umt-msba.transactions.transArchive_*`
    
"""
# Execute the query
query_job = client.query(query)

In [ ]:
query = """
    SELECT DISTINCT card_no
    FROM `umt-msba.transactions.transArchive_*`
    
"""
try:
    query_job = client.query(query)  # Start the query job
    results = query_job.result()  # Wait for the query to finish

    # Check if there are any results
    if results.total_rows > 0:
        print(f"Query succeeded with {results.total_rows} rows.")
        for row in results:
            len(row)  # Print each row, or handle results as needed
    else:
        print("Query executed but returned no results.")
except Exception as e:
    print(f"Query failed: {e}")

In [3]:
query = """
    SELECT DISTINCT card_no
    FROM `umt-msba.transactions.transArchive_*`
"""

# Execute the query
try:
    query_job = client.query(query)  # Start the query job
    results = query_job.result()  # Wait for the query to finish

    # Check if there are any results
    total_rows = results.total_rows  # Get the total number of rows
    if total_rows > 0:
        print(f"Query succeeded with {total_rows} rows.")
    else:
        print("Query executed but returned no results.")
except Exception as e:
    print(f"Query failed: {e}")

Query succeeded with 27208 rows.


In [12]:
results = query_job.result()



In [6]:
results_df = results.to_dataframe()


c:\Users\npleg\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.


In [7]:
type(results_df)

pandas.core.frame.DataFrame

In [8]:
results_df.to_csv("test.txt", sep="\t", index = False)

#Careful!!!!
--sample only from one set of data.
-- not sample all records for card_no....and make sure have all records for that card no. not just one record--- don't sample by row
